In [1]:
!pip install transformers datasets evaluate accelerate datasets transformers[torch]

In [23]:
!pip install accelerate -U

In [24]:
!pip install transformers[torch]

Load IMDb dataset

In [5]:
from datasets import load_dataset

imdb = load_dataset("imdb")

In [25]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [9]:
imdb["test"][0].keys()

dict_keys(['text', 'label'])

In [10]:
imdb["test"][0]['text']

'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they have

In [11]:
imdb["test"][0]['label']

0

Load the model from Huggingface

In [30]:
id2label = {1: "NEGATIVE", 0: "POSITIVE"}
label2id = {"NEGATIVE": 1, "POSITIVE": 0}

model_name='distilbert/distilbert-base-uncased'

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


reprocessing function to tokenize text and truncate sequences to be no longer than DistilBERT’s maximum input

In [31]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_imdb = imdb.map(preprocess_function, batched=True)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [32]:
import numpy as np



import evaluate

accuracy = evaluate.load("accuracy")


import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [33]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="checkpoint_folder",
    learning_rate=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"].select(range(5000)),
    eval_dataset=tokenized_imdb["test"].select(range(1000)),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model('full_finetune_model')

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000000,1.000000


In [34]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model_name)
classifier(text)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.512076735496521}]

In [35]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
from transformers import pipeline
finetune_model='full_finetune_model'
classifier = pipeline("sentiment-analysis", model=finetune_model)
classifier(text)

[{'label': 'POSITIVE', 'score': 1.0}]

In [36]:
ls -ltr checkpoint_folder

total 8
drwxr-xr-x  2 root root 4096 Jun  7 14:03 checkpoint-313/
drwxr-xr-x 12 root root 4096 Jun  7 14:05 runs/


In [37]:
ls -ltr full_finetune_model

total 262500
-rw-r--r-- 1 root root       746 Jun  7 14:09 config.json
-rw-r--r-- 1 root root 267832560 Jun  7 14:09 model.safetensors
-rw-r--r-- 1 root root      1196 Jun  7 14:09 tokenizer_config.json
-rw-r--r-- 1 root root       125 Jun  7 14:09 special_tokens_map.json
-rw-r--r-- 1 root root    231508 Jun  7 14:09 vocab.txt
-rw-r--r-- 1 root root    711494 Jun  7 14:09 tokenizer.json
-rw-r--r-- 1 root root      5112 Jun  7 14:09 training_args.bin
